In [ ]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import ensemble
from sklearn.tree import ExtraTreeRegressor
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn import ensemble
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
from sklearn.model_selection import LeaveOneOut
seed=3789
###########import packages##########
import catboost
import xgboost
import lightgbm
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import *
import pickle
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import  *
###########import packages##########
import tensorflow as tf
import keras
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.models import Model
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
# from keras.wrappers import scikit_learn
from scikeras.wrappers import KerasClassifier, KerasRegressor
###########loading data##########
loo = LeaveOneOut()
# %matplotlib



In [ ]:
###########wrapping root mean square error for later calls##########
def compute_mae_mse_rmse(target,prediction):
    error = []
    for i in range(len(target)):
        error.append(target[i] - prediction[i])
    squaredError = []
    absError = []
    for val in error:
        squaredError.append(val * val)  # target-prediction之差平方
        absError.append(abs(val))  # 误差绝对值
    mae=sum(absError)/len(absError)  # 平均绝对误差MAE
    mse=sum(squaredError)/len(squaredError)  # 均方误差MSE
    RMSE=np.sqrt(sum(squaredError)/len(squaredError))
    R2=r2_score(target,prediction)
    return mae,mse,RMSE,R2
def gridsearch(model,param,algorithm_name,X_train,y_train,X_test,y_test):
    grid = GridSearchCV(model,param_grid=param,scoring='neg_mean_absolute_error',cv=10,n_jobs=8,verbose=-1)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    ####Train####
    print("=========Train===========")
    prediction_train = best_model.predict(X_train)
    real_train=y_train.values
    prediction_train_series=pd.Series(prediction_train)
    real_train_series=pd.Series(real_train)
    corr_ann_train = round(prediction_train_series.corr(real_train_series), 5)
    error_val_train= compute_mae_mse_rmse(prediction_train,real_train)
    print(error_val_train)
    ####test####
    print("=========Test===========")
    prediction_test = best_model.predict(X_test)
    real_test=y_test.values
    prediction_test_series=pd.Series(prediction_test)
    real_test_series=pd.Series(real_test)
    corr_ann_test = round(prediction_test_series.corr(real_test_series), 5)
    error_val_test= compute_mae_mse_rmse(prediction_test,real_test)
    print(error_val_test)
    ####All####
    print("=========All===========")
    prediction_all = best_model.predict(data_input_full)
    real_all=data_output_full.values
    prediction_all_series=pd.Series(prediction_all)
    real_all_series=pd.Series(real_all)
    corr_ann_all = round(prediction_all_series.corr(real_all_series), 5)
    error_val_all= compute_mae_mse_rmse(prediction_all,real_all)
    print(error_val_all)
    
    print(algorithm_name)
    best_score=grid.best_score_
    print('Best Regressor:',grid.best_params_,'Best Score:', best_score)
    print('R2 TEST',error_val_test[3])
    fig=plt.figure()
    ax = fig.add_subplot(111)
    x_y_x=np.arange(-4,4,0.01)
    x_y_y=np.arange(-4,4,0.01)
    ax.scatter(prediction_train,real_train,c='blue',label='Train',alpha=0.25)
    ax.scatter(prediction_test,real_test,c='red',label='Test',alpha=0.75)
    ax.plot(x_y_x,x_y_y,c='black')
    plt.legend()
    plt.xlabel('Predicted_Overpotential@10 mA cm-2')
    plt.ylabel('Real_Overpotential@10 mA cm-2')
    return best_model,error_val_test[3]

In [ ]:
fl = open(r'./database_high_quality_st.pkl','rb')
database_full=pickle.load(fl)

In [ ]:
data_input_full=database_full.iloc[:,0:55]
data_output_full=database_full.iloc[:,55]
X_train,X_test,y_train,y_test=train_test_split(data_input_full,data_output_full,test_size=0.1,random_state=seed)

In [ ]:
model_CatRegressor=catboost.CatBoostRegressor(random_state=1,verbose=0)
param_cat = {
# 'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2],
# 'n_estimators':[100,200,400],
# "boosting_type":["Plain"],
# 'max_depth':[5,7,9,11],
# 'subsample':[0.4,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
# 'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
    'boosting_type': ['Plain'], 'learning_rate': [0.04], 'max_depth': [5], 'n_estimators': [400], 'reg_lambda': [0.001], 'subsample': [0.65]
}
CAT_full,CAT_full_score=gridsearch(model_CatRegressor,param_cat,'CatBoost',X_train,y_train,X_test,y_test)

In [ ]:
model_XGRegressor=XGBRegressor(random_state=1)
param_xg={
# 'booster':['gbtree'],
# 'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
# 'n_estimators':[100,200,400],
# 'max_depth':[3,5,7,9,11,13,-1],
# 'subsample':[0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
# 'reg_alpha':[0,0.001,0.01,0.0001,0.00001],
# 'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
    'booster': ['gbtree'], 'learning_rate': [0.01], 'max_depth': [13], 'n_estimators': [200], 'reg_alpha': [0.001], 'reg_lambda': [0.01], 'subsample': [0.5]
}
XG_full,XG_full_score=gridsearch(model_XGRegressor,param_xg,'XGBoost',X_train,y_train,X_test,y_test)

In [ ]:
###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestRegressor = ensemble.RandomForestRegressor(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
# 'n_estimators':[50,100,200,400,None],
# 'max_depth':[3,5,7,9,11,None],
# 'criterion':['mse','mae'],
# 'max_features':['auto','sqrt','log2']
'criterion': ['mae'], 'max_depth': [5], 'max_features': ['auto'], 'n_estimators': [100]
}
RF_full,RF_full_score=gridsearch(model_RandomForestRegressor,param_RF,'Random Forest',X_train,y_train,X_test,y_test)

In [ ]:
def predict_func_DC_400(p):
    ele_1,ele_2,ele_3,ele_4,prop_1,prop_2,prop_3,prop_4,hydro_temp,hydro_time,hydro_reduc,hydro_ball,post_process=p
    test_info=[0.5,0,1,1,1,10,36]
    ele_info=element_information[ele_1]+element_information[ele_2]+element_information[ele_3]+element_information[ele_4]
    prop_info=[prop_1,prop_2,prop_3,prop_4]
    hydrothermal_info=[hydro_temp,hydro_time,1]+[0,hydro_reduc,hydro_ball]
    annealing_info=[400,360,0,0,0,post_process]
    ###合并制备条件
    info_all=ele_info+prop_info+hydrothermal_info+annealing_info+test_info
    input_array=np.array(info_all)
    input_array=input_array.reshape(1, -1)
    input_array=pd.DataFrame(input_array,columns=data_input_full.columns)
    #####预测
    pred_list=[]
#     pred_list.append(XG_full.predict(input_array)[0])
    pred_list.append(CAT_full.predict(input_array)[0])
#     pred_list.append(RF_full.predict(input_array)[0])
    ####weighted_score####
    test_score_list=[
#     XG_full_score,
    CAT_full_score
#     RF_full_score
    ]

    pred_array = np.array(pred_list)
    test_score_array = np.array(test_score_list)
    
    # Calculate the weighted result
    weighted_result = np.dot(pred_array, test_score_array) / np.sum(test_score_array)
    print(weighted_result)
    return weighted_result
def predict_func_DC_500(p):
    ele_1,ele_2,ele_3,ele_4,prop_1,prop_2,prop_3,prop_4,hydro_temp,hydro_time,hydro_reduc,hydro_ball,post_process=p
    test_info=[0.5,0,1,1,1,10,36]
    ele_info=element_information[ele_1]+element_information[ele_2]+element_information[ele_3]+element_information[ele_4]
    prop_info=[prop_1,prop_2,prop_3,prop_4]
    hydrothermal_info=[hydro_temp,hydro_time,1]+[0,hydro_reduc,hydro_ball]
    annealing_info=[500,360,0,0,0,post_process]
    ###合并制备条件
    info_all=ele_info+prop_info+hydrothermal_info+annealing_info+test_info
    input_array=np.array(info_all)
    input_array=input_array.reshape(1, -1)
    input_array=pd.DataFrame(input_array,columns=data_input_full.columns)
    #####预测
    pred_list=[]
#     pred_list.append(XG_full.predict(input_array)[0])
    pred_list.append(CAT_full.predict(input_array)[0])
#     pred_list.append(RF_full.predict(input_array)[0])
    ####weighted_score####
    test_score_list=[
#     XG_full_score,
    CAT_full_score
#     RF_full_score
    ]
    
    pred_array = np.array(pred_list)
    test_score_array = np.array(test_score_list)

    # Calculate the weighted result
    weighted_result = np.dot(pred_array, test_score_array) / np.sum(test_score_array)
    print(weighted_result)
    return weighted_result


In [ ]:
mat_400_ALL=pd.read_csv('400.csv',header=None)
mat_500_ALL=pd.read_csv('500.csv',header=None)

In [ ]:
mat_400=mat_400_ALL.iloc[:,:-1]
mat_500=mat_500_ALL.iloc[:,:-1]

In [ ]:
element_information={}
element_information['None']=[0,0,0,0,0,0,0,0]
element_information['none']=[0,0,0,0,0,0,0,0]
element_information['Sc']=[44.956, 21, 4, 3, 6.57, 1.36, 1, 162]
element_information['Ti']=[47.867, 22, 4, 4, 6.81, 1.54, 2, 147]
element_information['V']=[50.942, 23, 4, 5, 6.76, 1.63, 3, 134]
element_information['Cr']=[51.996, 24, 4, 6, 6.74, 1.66, 4, 130]
element_information['Mn']=[54.938, 25, 4, 7, 7.4, 1.56, 5, 127]
element_information['Fe']=[55.845, 26, 4, 8, 7.83, 1.83, 6, 126]
element_information['Co']=[58.933, 27, 4, 9, 7.81, 1.88, 7, 125]
element_information['Ni']=[58.6934, 28, 4, 10, 7.61, 1.91, 8, 124]
element_information['Cu']=[63.546, 29, 4, 11, 7.69, 1.9, 10, 128]
element_information['Zn']=[65.38, 30, 4, 12, 9.35, 1.65, 10, 138]
element_information['Ga']=[69.723, 31, 4, 13, 5.97, 1.81, 10, 141]
element_information['Y']=[88.905, 39, 5, 3, 6.5, 1.22, 1, 178]
element_information['Zr']=[91.224, 40, 5, 4, 6, 1.33, 2, 160]
element_information['Nb']=[92.906, 41, 5, 5, 6.76, 1.6, 4, 146]
element_information['Mo']=[95.94, 42, 5, 6, 7.35, 2.16, 5, 139]
element_information['Tc']=[98.906, 43, 5, 7, 7.28, 1.9, 5, 136]
element_information['Ru']=[101.07, 44, 5, 8, 7.5, 2.2, 7, 134]
element_information['Rh']=[102.905, 45, 5, 9, 7.7, 2.28, 8, 134]
element_information['Pd']=[106.42, 46, 5, 10, 8.3, 2.2, 10, 137]
element_information['Ag']=[107.868, 47, 5, 11, 7.54, 1.93, 10, 144]
element_information['Cd']=[112.411, 48, 5, 12, 8.95, 1.69, 10, 154]
element_information['In']=[114.818, 49, 5, 13, 8.95, 1.78, 10, 166]
element_information['Sn']=[118.71, 50, 5, 14, 7.37, 1.96, 10, 162]
element_information['La']=[138.905, 57, 6, 3, 5.5, 1.1, 1, 187]
element_information['Ce']=[140.116, 58, 6, 3, 6.91, 1.12, 1, 181]
element_information['Pr']=[140.904, 59, 6, 3, 5.76, 1.13, 10, 182]
element_information['Nd']=[144.242, 60, 6, 3, 6.31, 1.14, 10, 182]
element_information['Pm']=[144.912, 61, 6, 3 ,5.55, 1.13, 10, 183]
element_information['Sm']=[150.36, 62, 6, 3, 6.55, 1.1, 10, 181]
element_information['Eu']=[151.964, 63, 6, 3, 5.67, 1.2, 10, 199]
element_information['Gd']=[157.25, 64, 6, 3, 6.65, 1.2, 1, 179]
element_information['Tb']=[158.925, 65, 6, 3, 6.74, 1.2, 10, 180]
element_information['Dy']=[162.5, 66, 6, 3, 6.82, 1.22, 10, 180]
element_information['Ho']=[164.93, 67, 6, 3, 6.022, 1.23, 10, 179]
element_information['Er']=[167.529, 68, 6, 3, 6.108, 1.23, 10, 178]
element_information['Tm']=[168.934, 69, 6, 3, 6.184, 1.25, 10, 177]
element_information['Yb']=[173.04, 70, 6, 3, 7.06, 1.1, 10, 176]
element_information['Lu']=[174.967, 71, 6, 3, 5.4259, 1.27, 1, 175]
element_information['Hf']=[178.49, 72, 6, 4, 6.8251, 1.3, 2, 167]
element_information['Ta']=[180.947,73, 6, 5, 7.89, 1.5, 3, 149]
element_information['W']=[183.84, 74, 6, 6, 7.98, 2.36, 4, 141]
element_information['Re']=[186.207, 75, 6, 7, 7.88, 1.9, 5, 137]
element_information['Os']=[190.23, 76, 6, 8, 8.7, 2.2, 6, 135]
element_information['Ir']=[192.217, 77, 6, 9, 9.1, 2.2, 7, 136]
element_information['Pt']=[195.084, 78, 6, 10, 8.9, 2.28, 9, 139]
element_information['Au']=[196.966, 79, 6, 11, 9.19, 2.54, 10, 144]
element_information['Hg']=[200.59, 80, 6, 12, 10.39, 2, 10, 157]
element_information['Tl']=[204.383, 81, 6, 13, 6.08, 1.62, 10, 171]
element_information['Pb']=[207.2, 82, 6, 14, 7.38, 2.33, 10, 175]
element_information['Bi']=[208.98, 83, 6, 15, 7.25, 2.02, 10, 170]
###added
element_information['Ba']=[137.327, 56, 6, 2, 5.19, 0.89, 10, 222]
element_information['Sr']=[87.62, 38, 5, 2, 5.67, 0.95, 10, 215]
element_information['Na']=[22.9897, 11, 3, 1, 5.12, 0.93, 0, 190]
element_information['K']=[39.0983, 19, 4, 1, 4.32, 0.82, 0, 235]
element_information['Ca']=[40.078, 20, 4, 2, 6.09, 1, 0, 197]
element_information['Mg']=[24.3050, 12, 3, 2, 7.61, 1.31, 0, 160]
element_information['Li']=[6.941, 3, 2, 1, 5.37, 0.98, 0, 145]
element_information['C']=[12.0107, 6, 2, 14, 11.22, 2.56, 0, 77]
element_information['B']=[10.811, 5, 2, 13, 8.33, 2.04, 0, 98]
element_information['N']=[14.0067, 7, 2, 15, 14.48, 3.04, 0, 92]
element_information['P']=[30.9737, 15, 3, 15, 10.3, 2.19, 0, 128]
element_information['F']=[18.9984, 9, 2, 17, 18.6, 3.98, 0, 73]
element_information['S']=[32.065, 16, 3, 16, 10.31, 2.58, 0, 127]
element_information['Sb']=[121.760, 51, 5, 15, 8.35, 2.05, 10, 159]
element_information['Te']=[127.6, 52, 52, 5, 16, 9.0096, 2.1, 10, 160]
element_information['Br']=[79.904, 35, 4, 17, 11.8, 2.96, 10, 115]
element_information['Cl']=[35.453, 17, 3, 17, 12.96, 3.16, 0, 99]
element_information['Si']=[28.0855, 14, 3, 14, 8.12, 1.9, 0, 132]
element_information['Se']=[78.96, 34, 4, 16, 9.5, 2.55, 10, 140]
element_information['Al']=[26.9815, 13, 3, 13, 5.95, 1.61, 0,143]

In [ ]:
def replace_last_element(tuple_data, new_value):
    # 将元组转换为列表
    list_data = list(tuple_data)
    
    # 替换最后一个元素
    list_data[-1] = new_value
    
    # 将列表转换回元组
    new_tuple = tuple(list_data)
    
    return new_tuple

In [ ]:
def compute_long_time(temp,mat_ALL,time):
    init_OVP=mat_ALL.iloc[i,-1]
    if temp==400:
        DC_RATE=predict_func_DC_400(replace_last_element(tuple(mat_400.iloc[i]),time))
    elif temp==500:
        DC_RATE=predict_func_DC_500(replace_last_element(tuple(mat_500.iloc[i]),time))
    else:
        print('wrong temp')
    print(DC_RATE)
    final_OVP=init_OVP+(10**DC_RATE)*time
    print(init_OVP,final_OVP)
    return init_OVP,final_OVP

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Function to add labels to the top two bars
def add_labels(ax, values):
    sorted_values = sorted(values)
    for i in range(2):  # For the smallest two values
        index = values.index(sorted_values[i])
        ax.text(index, sorted_values[i], f'{sorted_values[i]:.2f}', ha='center', fontsize=8)

# Function to assign colors based on value
def assign_colors(values):
    max_val = max(values)
    min_val = min(values)
    # Assign colors based on the value - darker for smaller values, lighter for larger values
    return [plt.cm.Blues((max_val - value) / (max_val - min_val)) for value in values]

plt.figure(figsize=(15, 5))

# Plot for 400 ℃ Products
ax1 = plt.subplot(1, 2, 1)
colors_400 = assign_colors(sorted(sum_400_list))
ax1.bar(range(len(sum_400_list)), sorted(sum_400_list), color=colors_400)
ax1.set_xlabel('Sample Serial Number-400 ℃ Products')
ax1.set_ylabel('ML Predicted Electricity Consumed over 36 h (W)')
ax1.set_title('Ranking of Electricity Consumed over 36 h (W)-400 ℃ Products')
ax1.set_xticks(range(len(sum_400_list)))
ax1.set_xticklabels([f'{i+1}th' for i in sum_400_ranking], rotation=45)
ax1.tick_params(axis='x', direction='in')
add_labels(ax1, sorted(sum_400_list))

# Plot for 500 ℃ Products
ax2 = plt.subplot(1, 2, 2)
colors_500 = assign_colors(sorted(sum_500_list))
ax2.bar(range(len(sum_500_list)), sorted(sum_500_list), color=colors_500)
ax2.set_xlabel('Sample Serial Number-500 ℃ Products')
ax2.set_ylabel('ML Predicted Electricity Consumed over 36 h (W)')
ax2.set_title('Ranking of Electricity Consumed over 36 h (W)-500 ℃ Products')
ax2.set_xticks(range(len(sum_500_list)))
ax2.set_xticklabels([f'{i+1}th' for i in sum_500_ranking], rotation=45)
ax2.tick_params(axis='x', direction='in')
add_labels(ax2, sorted(sum_500_list))

plt.tight_layout()
plt.show()
